<a href="https://colab.research.google.com/github/LucianoBV/Procesamiento-del-habla/blob/main/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


1) Crear un dataset de preguntas y respuestas para crear un Chatbot.

In [109]:
df = pd.read_excel("data set chat-bot.xlsx")
df

,ID,Pregunta,Respuesta
0,1,¿Cuáles son los horarios de atención del juzgado?,"El juzgado atiende de lunes a viernes, de 8:00..."
1,2,¿Dónde está ubicado el juzgado?,El juzgado está ubicado en la Calle Principal ...
2,3,¿Qué documentos necesito para presentar una de...,Los documentos que necesitas son: identificaci...
3,4,¿Cómo puedo agendar una cita?,"Para agendar una cita, puedes hacerlo a través..."
4,5,¿Cuáles son los tipos de juicios que se pueden...,En este juzgado se pueden presentar juicios ci...
5,6,¿Qué hago si no puedo asistir a una audiencia?,"Si no puedes asistir, debes informar al juzgad..."
6,7,¿Cómo puedo consultar el estado de mi caso?,Puedes consultar el estado de tu caso en línea...
7,8,¿Cuál es el procedimiento para solicitar una o...,Debes acudir al juzgado con pruebas del peligr...
8,9,¿Puedo obtener asesoría legal gratuita en el j...,"Sí, contamos con asesoría legal gratuita para ..."
9,10,¿Cuáles son los días festivos en los que el ju...,El juzgado está cerrado los días festivos naci...


2) Crear el chatbot utilizando TFIDF y similitud del coseno.

In [110]:

# extraer las preguntas y respuestas
pregunta = df["Pregunta"].values
respuesta = df["Respuesta"].values

# Iniciar el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Ajustar y transformar las preguntas en vectores TF-IDF.
tfidf_matrix = vectorizer.fit_transform(pregunta)

# Definir una función para procesar una consulta de usuario y devolver la respuesta más similar
def obtener_respuesta(consulta_del_usuario):
    # Transforme la consulta del usuario en un vector TF-IDF
    pregunta_vec = vectorizer.transform([consulta_del_usuario])

    # Calcule la similitud del coseno entre la consulta del usuario y todas las preguntas
    coseno_similitudes = cosine_similarity(pregunta_vec, tfidf_matrix).flatten()

    # Encuentra el índice de la pregunta más similar.
    mejor_índice_de_coincidencia = coseno_similitudes.argmax()

    # Devuelve la respuesta correspondiente
    return respuesta[mejor_índice_de_coincidencia], coseno_similitudes[mejor_índice_de_coincidencia]

# Pruebe el sistema con una consulta de muestra
consulta_prueba = "¿Cómo puedo agendar una cita?"
respuesta_del_bot = obtener_respuesta(consulta_prueba)
print(respuesta_del_bot)

('Para agendar una cita, puedes hacerlo a través del portal web ', np.float64(1.0000000000000002))


3) Crear otro chatbot utilizando embeddings. Indique cuál embedding pre-entrenado eligió.

In [111]:
!pip install spacy
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [112]:
import spacy

# Cargar el modelo en español con vectores preentrenados
nlp = spacy.load("es_core_news_md")

In [113]:
preguntas= [nlp(pregunta) for pregunta in df['Pregunta']]

In [114]:
def preguntas_aca(consultoria):
    consultoria=nlp(consultoria)
    coincidencias = [pregunta.similarity(consultoria) for pregunta in preguntas]
    respuesta_correcta = df['Respuesta'][coincidencias.index(max(coincidencias))]
    return respuesta_correcta
que_necesita="¿Cuáles son los horarios de atención del juzgado?"
preguntas_aca(que_necesita)

'El juzgado atiende de lunes a viernes, de 8:00 a 15:00 horas.'

Enfoque y aproximación ⚙️💡
Para crear estos chatbots, utilicé dos enfoques principales de procesamiento de lenguaje natural: TFIDF con similitud de coseno y embeddings. Ambos tienen ventajas y desventajas, por lo que es útil probarlos para comparar rendimiento y precisión en escenarios prácticos. 🔄

Chatbot con TFIDF y similitud de coseno: 🧮
Convierte textos en vectores numéricos basados en la frecuencia de términos. Luego, uso la similitud de coseno para encontrar preguntas similares. Es rápido ⚡ y funciona bien con datos moderados y preguntas conocidas, pero no captura el contexto ni las relaciones complejas entre palabras. Si la pregunta es diferente a las del dataset, puede fallar ❌.

Chatbot con embeddings: 🧠
Los embeddings permiten representar frases de forma más profunda, capturando el contexto y significado. 📖🔍

Comparación:

Rendimiento: TFIDF es más rápido y eficiente 🏃‍♂️, pero su precisión disminuye con preguntas variadas 📉.
Precisión: Los embeddings son más precisos, entendiendo mejor el significado detrás de las palabras 🎯✨.
En mi experiencia, TFIDF es ideal para soluciones rápidas ⚙️, mientras que los embeddings mejoran la interacción con respuestas más precisas 🤖👍.